In [1]:
%load_ext autoreload
%autoreload 2

# 1. Media Creation Example

This example shows how to create a media object from a file and upload it to the server. It save the metadata in the postgres db and uploads the files to the s3 bucket.

In [2]:
from rts.db.dao import DataAccessObject
from rts.db_settings import DATABASE_URL
from rts.db.queries import get_library_id_from_name, read_media_by_source_media_id
from rts.io.media import upload_media_files
from rts.api.models import Media
import os
import hashlib
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
DataAccessObject().connect(DATABASE_URL)

ARCHIVE_BASE_PATH = os.getenv("BASE_PATH")

/Users/arattinger/Projects/rts/rts/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Setup before we can create media objects
archive_name =  "rts"
bucket_name =  archive_name
library_id =  get_library_id_from_name(archive_name)
library_id

1

In [5]:
clips = []
data_dir = os.path.join(ARCHIVE_BASE_PATH, "archive")

count = 0
archive_path = os.path.join(ARCHIVE_BASE_PATH, "archive")
print(archive_path)
for dirpath, dirnames, filenames in os.walk(archive_path):
    if 'clips' not in dirnames:
        continue
    
    clip_dir = os.path.join(dirpath, 'clips', 'videos')

    for clip in os.listdir(clip_dir):
        clip_path = os.path.join(clip_dir, clip)
        try:
            dirname = dirpath.split('/')[-1]

            clips.append(Media(**{
                    'media_id': clip.split('.')[0],
                    'original_path': clip_path,
                    'original_id': dirpath.split("/")[-1],
                    'media_path': f"videos/{dirname}/{clip}", 
                    'media_type': "video",
                    'sub_type': "clip", 
                    'size': os.path.getsize(clip_path), 
                    'metadata': {},
                    'library_id': library_id, 
                    'hash': hashlib.md5(f"{bucket_name}/videos/{dirname}/{clip}".encode()).hexdigest(), 
                    'parent_id': -1,
                    'start_ts': 0, 'end_ts': 10, 
                    'start_frame': 0, 'end_frame': 10, 'frame_rate': 30, 
            }))

        except NotADirectoryError:
            pass

    count += 1
    if count > 5:
        break    
len(clips)

/Users/arattinger/Projects/rts/rts/testdata/archive


21

In [6]:
uploaded_clips = upload_media_files(clips)

In [7]:
thumbnails = []
square_res = 256
media_keys = set()

for clip in uploaded_clips:
    clip_rel_dir = os.path.join('clips', 'images', '256px')
    clip_dir = os.path.join(archive_path, clip['original_id'], clip_rel_dir)
    try:
        for img in os.listdir(clip_dir):

            img_path = os.path.join(clip_dir, img)
            media_path = f"images/{dirname}/{square_res}px/{img}"

            if media_path in media_keys:
                continue

            thumbnails.append(Media(**{
                    'media_id': img.split('.')[0],
                    'original_path': img_path,
                    'original_id': clip['original_id'],
                    'media_path': media_path, 
                    'media_type': "image",
                    'sub_type': "thumbnail", 
                    'size': os.path.getsize(clip_path),
                    'metadata': {},
                    'library_id': library_id, 
                    'hash': hashlib.md5(f"videos/{dirname}/{img}".encode()).hexdigest(), 
                    'parent_id': clip['media_id'],
                    'start_ts': 0, 'end_ts': 10, 
                    'start_frame': 0, 'end_frame': 10, 'frame_rate': 30, 
            }))
            media_keys.add(media_path)

    except FileNotFoundError:
        print(f"Directory not found: {clip_dir}")
        continue
len(thumbnails)

63

In [8]:
uploaded_thumbnails = upload_media_files(thumbnails)